In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import TokenTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import json
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain



In [3]:

load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')

In [4]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

cache_store = LocalFileStore("./cache/")

In [5]:
loader = DirectoryLoader('../data', glob="*.txt", loader_cls=TextLoader, show_progress=True)
docs = loader.load()
# pretty_print_docs(docs)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1418.67it/s]


In [6]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=128,
    chunk_overlap=30,
)
chunked = text_splitter.split_documents(docs)
# print(len(chunked))
# pretty_print_docs(chunked)

In [7]:
cache_store = LocalFileStore("./nomic_cache/")

In [8]:
embeddings_model = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [9]:
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_model, cache_store, namespace=embeddings_model.model)

In [10]:
%%time
db = FAISS.from_documents(chunked, cached_embedder)

CPU times: user 1.07 s, sys: 308 ms, total: 1.38 s
Wall time: 9.78 s


In [11]:
llm = Ollama(model="mistral", temperature=0)
query = "How do I defeat Dreki?"

In [12]:
retriever = db.as_retriever()
docs = retriever.get_relevant_documents(query)
pretty_print_docs(docs)

Document 1:

Once you have managed to deplete the Dreki’s health bar, press R3 when prompted to finally defeat the creature with the help of Atreus. Defeating the Dreki will reward you with a Chaos Flame, +4 Dragon Tooths, and a Light Runic Attack (Flames of Anguish).
----------------------------------------------------------------------------------------------------
Document 2:

Straight out of the gate, the Dreki will spit a ball of saliva toward you – you’ll be dealt damage if you do not move in time. And while it’s arguably easier to just dodge this attack, it’s also possible to block it with your shield, or even parry it back into the creature if you’re using the Dauntless Shield, dealing damage and temporarily stunning it.
----------------------------------------------------------------------------------------------------
Document 3:

Once collected, follow Atreus north through a small hole that’s found in the rock wall, where you’ll discover a fallen Dreki.

Defeat Dreki
While i

In [13]:
%%time
print(llm.invoke(query))

 Dreki is a dragon boss found in the Elder Scrolls V: Skyrim. Defeating Dreki requires preparation, strategy, and knowledge of its strengths and weaknesses. Here are some steps to help you defeat Dreki:

1. Preparation:
   - Ensure your character is well-equipped with appropriate armor, weapons, and potions.
   - Make sure you have a good selection of spells if you're using magic.
   - Save the game before attempting to fight Dreki.

2. Location:
   - Find Dreki's lair in the northern part of Skyrim, near the Throat of the World.
   - Approach the lair cautiously, as there may be dragons guarding the entrance.

3. Strategy:
   - Use ranged attacks or spells to deal damage from a safe distance.
   - If you're using melee, try to avoid getting too close to Dreki's breath attack.
   - Focus on damaging Dreki's weak spots: its wings and head.
   - Use potions and spells to heal yourself during the fight.

4. Tactics:
   - If you have a follower, consider bringing them along for additional 

In [14]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(query)
pretty_print_docs(compressed_docs)

Document 1:

Once collected, follow Atreus north through a small hole that’s found in the rock wall, where you’ll discover a fallen Dreki.

Defeat Dreki
While inspecting the Dreki, a larger Dreki will appear and charge toward both Atreus and Kratos. The large crocodile-like creature will consist of five sectioned health bars, so you’ll need to deal quite a bit of damage to defeat the monster.
----------------------------------------------------------------------------------------------------
Document 2:

Once you have managed to deplete the Dreki’s health bar, press R3 when prompted to finally defeat the creature with the help of Atreus. Defeating the Dreki will reward you with a Chaos Flame, +4 Dragon Tooths, and a Light Runic Attack (Flames of Anguish).
----------------------------------------------------------------------------------------------------
Document 3:

Straight out of the gate, the Dreki will spit a ball of saliva toward you – you’ll be dealt damage if you do not move in

In [15]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever
)


CPU times: user 712 µs, sys: 972 µs, total: 1.68 ms
Wall time: 1.98 ms


In [16]:
%%time
c2 = chain({"query": query})

/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: user 53.3 ms, sys: 41.6 ms, total: 94.9 ms
Wall time: 6.09 s


In [17]:
print(c2["result"])

 To defeat Dreki in God of War, you need to deplete its five health bars by dealing significant damage. When prompted, press R3 to help Atreus finish off the creature. During the battle, be aware that Dreki will spit a ball of saliva at you, which can be dodged or blocked with your shield. If you're using the Dauntless Shield, you can also parry it back and deal damage to the creature.


In [18]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=Cohere(temperature=0), retriever=compression_retriever, chain_type="refine"
)
c3 = chain({"query": query})

/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.cohere.Cohere` was deprecated in langchain-community 0.1.14 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


CPU times: user 59.7 ms, sys: 7.38 ms, total: 67.1 ms
Wall time: 8.92 s


In [19]:
print(c3["result"])

 To defeat Dreki you need to deal a lot of damage to different parts of its body while avoiding its attacks. Dreki will spit a ball of saliva toward you – you’ll be dealt damage if you do not move in time, but it is also possible to block it with your shield, or even parry it back into the creature if you’re using the Dauntless Shield to deal damage and temporarily stun it. Once its health bar is depleted you need to press R3 when prompted in order to finally defeat it with Atreus' help. Beating Dreki will reward you with various items and a new runic attack. 


In [20]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever, chain_type="refine"
)
c4 = chain({"query": query})

CPU times: user 169 ms, sys: 73.6 ms, total: 242 ms
Wall time: 22.7 s


In [21]:
print(c4["result"])

 To defeat Dreki as described in the context information, follow Atreus north through a small hole in the rock wall where you'll find a fallen Dreki. When inspecting the Dreki, a larger Dreki will emerge and charge towards both Atreus and Kratos. This larger Dreki consists of five sectioned health bars, so to defeat it, you'll need to deal significant damage to the creature multiple times using your available weapons and abilities.

When the Dreki spits a ball of saliva at you, you have several options to respond: you can dodge it by moving out of the way, block it with your shield, or even parry it back into the creature using the Dauntless Shield, dealing damage and temporarily stunning it. This can be an effective strategy to gain an advantage during the battle.

Once all of its health bars are depleted, press R3 when prompted to finally defeat the creature with the help of Atreus. Defeating the Dreki will reward you with a Chaos Flame, +4 Dragon Tooths, and a Light Runic Attack (Fl